# 载入数据

In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# 建立模型

In [2]:
# 定义占位符
x=tf.placeholder(tf.float32,[None,784],name="X")
y=tf.placeholder(tf.float32,[None,10],name="Y")

In [3]:
image_shaped_input=tf.reshape(x,[-1,28,28,1])
tf.summary.image('input',image_shaped_input,10)# 加入摘要图像信息

<tf.Tensor 'input:0' shape=() dtype=string>

# 构建隐藏层

In [4]:
def fcn_layer(inputs,      # 输入数据
             input_dim,   #输入神经元数量   
             output_dim,   #输出神经元数量 
             activation=None): #激活函数
    W=tf.Variable(tf.truncated_normal([input_dim,output_dim],stddev=0.1))
    b=tf.Variable(tf.zeros(output_dim))
    XWb=tf.matmul(inputs,W)+b
    if activation is None:
        outputs=XWb
    else:
        outputs=activation(XWb)
    return outputs
              

In [5]:
# 隐藏层神经元数量
H1_NN=256 
H2_NN=64


# 构建隐藏层一

In [6]:
h1=fcn_layer(inputs=x,input_dim=784,output_dim=H1_NN,activation=tf.nn.relu)

# 构建隐藏层二

In [7]:
h2=fcn_layer(inputs=h1,input_dim=H1_NN,output_dim=H2_NN,activation=tf.nn.relu)

# 构建输出层

In [8]:
forward=fcn_layer(inputs=h2,input_dim=H2_NN,output_dim=10,activation=None)
pred=tf.nn.softmax(forward)

# Tensorboard：前向计算直方图显示

In [9]:
tf.summary.histogram('forward',forward)

<tf.Tensor 'forward:0' shape=() dtype=string>

# 训练模型

In [10]:
# 交叉熵
#loss_function=tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred),reduction_indices=1))

In [11]:

loss_function=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# 将loss以损失标量显示

In [13]:
tf.summary.scalar('loss',loss_function)

<tf.Tensor 'loss:0' shape=() dtype=string>

# 设置训练参数

In [14]:
train_epochs=40
batch_size=50
total_batch=int(mnist.train.num_examples/batch_size)
display_step=1
learning_rate=0.01

# 选择优化器

In [15]:
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

# 定义准确率

In [16]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(pred,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# 以accuracy准确率以标量

In [17]:
tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

# 训练模型

In [ ]:
# 记录训练开始时间
from time import time
startTime=time()
sess=tf.Session()
sess.run(tf.global_variables_initializer())
##合并所有summary
merged_summary_op=tf.summary.merge_all()
writer=tf.summary.FileWriter('log/',sess.graph)# 创建写入符

##
for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs,ys=mnist.train.next_batch(batch_size)# 读取批次数据
        sess.run(optimizer,feed_dict={x:xs,y:ys})#执行批次训练
        # total_batch个批次训练完成后，使用验证数据计算误差和准确率
        
        ##生成summary
        summary_str=sess.run(merged_summary_op,feed_dict={x:xs,y:ys})
        writer.add_summary(summary_str,epoch)# 将summary写入文件
        
        ##
        loss,acc=sess.run([loss_function,accuracy],feed_dict={x:mnist.validation.images,y:mnist.validation.labels})
    if (epoch+1)%display_step==0:
        print("Train Epoch:",'%02d'%(epoch+1),
                 "Loss=","{:.9f}".format(loss),"Accuracy=","{:.4f}".format(acc))
duration=time()-startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 Loss= 0.137077391 Accuracy= 0.9630
Train Epoch: 02 Loss= 0.166402325 Accuracy= 0.9588
Train Epoch: 03 Loss= 0.149445698 Accuracy= 0.9640
Train Epoch: 04 Loss= 0.141960323 Accuracy= 0.9650
Train Epoch: 05 Loss= 0.121554062 Accuracy= 0.9684
Train Epoch: 06 Loss= 0.159920454 Accuracy= 0.9666
Train Epoch: 07 Loss= 0.155972570 Accuracy= 0.9676
Train Epoch: 08 Loss= 0.145663574 Accuracy= 0.9692
Train Epoch: 09 Loss= 0.149761066 Accuracy= 0.9708
Train Epoch: 10 Loss= 0.144102395 Accuracy= 0.9730
Train Epoch: 11 Loss= 0.168708667 Accuracy= 0.9706
Train Epoch: 12 Loss= 0.163702026 Accuracy= 0.9740
Train Epoch: 13 Loss= 0.204716191 Accuracy= 0.9682
Train Epoch: 14 Loss= 0.176767796 Accuracy= 0.9702
Train Epoch: 15 Loss= 0.174972102 Accuracy= 0.9730
Train Epoch: 16 Loss= 0.248001680 Accuracy= 0.9708
Train Epoch: 17 Loss= 0.208040193 Accuracy= 0.9690
Train Epoch: 18 Loss= 0.154676050 Accuracy= 0.9702
Train Epoch: 19 Loss= 0.216201752 Accuracy= 0.9716
Train Epoch: 20 Loss= 0.2024870